In [1]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import time
import argparse
import numpy as np
import torch as t
import torch.nn as nn
import torchvision as tv
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np

import torch
from torch.utils.data import TensorDataset

from utils import get_train_test

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [3]:
import torch_fidelity

In [4]:
def norm_ip(img, min, max):
    temp = t.clamp(img, min=min, max=max)
    temp = (temp + -min) / (max - min + 1e-5)
    return temp

In [5]:
device = t.device('cuda' if t.cuda.is_available() else 'cpu')


In [6]:
args = argparse.Namespace()
args.dataset = 'cifar10'
args.data_root = '../../data'
args.seed = 1
data_loader, test_loader = get_train_test(args)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [7]:
from data import Cifar10
test_dataset = Cifar10(args, full=True, noise=False)

test_dataloader = DataLoader(test_dataset, batch_size=100, num_workers=0, shuffle=True, drop_last=False)

test_ims = []

def rescale_im(im):
    return np.clip(im * 256, 0, 255).astype(np.uint8)

for data_corrupt, data, label_gt in test_dataloader:
    if args.dataset in ['celeba128', 'img32']:
        data = data_corrupt.numpy().transpose(0, 2, 3, 1)
    else:
        data = data.numpy()
    test_ims.extend(list(rescale_im(data)))
    if (args.dataset == "imagenet" or 'img' in args.dataset) and len(test_ims) > 60000:
        test_ims = test_ims[:60000]
        break
        
print(np.array(test_ims).shape)

Files already downloaded and verified
Files already downloaded and verified
(60000, 32, 32, 3)


In [8]:
test_imgs = np.array(test_ims).transpose(0, 3, 1, 2)

In [9]:
train_images = []
for data, y, _  in data_loader:
    data = norm_ip(data, -1, 1).numpy()
    train_images.extend(list(rescale_im(data)))
    if (args.dataset == "imagenet" or 'img' in args.dataset) and len(train_images) > 60000:
        train_images = train_images[:60000]
        break
        
train_images = np.array(train_images)

In [10]:
test_images = []
for data, y, _ in test_loader:
    data = norm_ip(data, -1, 1).numpy()
    test_images.extend(list(rescale_im(data)))
    if (args.dataset == "imagenet" or 'img' in args.dataset) and len(test_images) > 60000:
        test_images = test_images[:60000]
        break
test_images = np.array(test_images)

In [11]:
test_imgs.shape

(60000, 3, 32, 32)

In [12]:
print(train_images.shape, test_images.shape)

(50000, 3, 32, 32) (10000, 3, 32, 32)


In [13]:
train_images.min(), test_images.min(), train_images.max(), test_images.max()

(0, 0, 255, 255)

# benchmarks

cifar10 train set, test set,  train_set[:10000]

In [14]:
class TensorDataset(Dataset):

    def __init__(self, x):
        self.x = x

    def __getitem__(self, index):
        return self.x[index]

    def __len__(self):
        return len(self.x)

In [15]:
start = time.time()
px_dataset = TensorDataset(torch.Tensor(test_images).to(dtype=torch.uint8))

metrics_dict = torch_fidelity.calculate_metrics(
    input1=px_dataset,
    input2=f'cifar10-train',
    cuda=True,
    isc=True,
    fid=True,
    # kid=True,
    verbose=False,
)
print("takes {}s".format(time.time() - start))
print(metrics_dict)
# tensorflow version's results:
# Obtained fid value of 2.203647314570105
# FID of score 2.203647314570105 takes 166.0104057788849s

takes 35.565284729003906s
{'inception_score_mean': 10.960357379524712, 'inception_score_std': 0.23457692862181354, 'frechet_inception_distance': 3.1508447258931938}


In [16]:
start = time.time()
px_dataset = TensorDataset(torch.Tensor(train_images).to(dtype=torch.uint8))

metrics_dict = torch_fidelity.calculate_metrics(
    input1=px_dataset,
    input2=f'cifar10-train',
    cuda=True,
    isc=True,
    fid=True,
    # kid=True,
    verbose=False,
)
print("takes {}s".format(time.time() - start))
print(metrics_dict)

# tensorflow version's results:
# Obtained fid value of 0.08768652773858321
# FID of score 0.08768652773858321 takes 254.37319445610046s

takes 93.05363821983337s
{'inception_score_mean': 11.233422048161026, 'inception_score_std': 0.20987301538533015, 'frechet_inception_distance': 4.3486920731083956e-09}


In [17]:
start = time.time()
px_dataset = TensorDataset(torch.Tensor(train_images[:10000]).to(dtype=torch.uint8))

metrics_dict = torch_fidelity.calculate_metrics(
    input1=px_dataset,
    input2=f'cifar10-train',
    cuda=True,
    isc=True,
    fid=True,
    # kid=True,
    verbose=False,
)
print("takes {}s".format(time.time() - start))
print(metrics_dict)

# tensorflow version's results:
# Obtained fid value of 2.192182121402425
# FID of score 2.192182121402425 takes 167.54650378227234s

takes 34.147802114486694s
{'inception_score_mean': 11.139391022878295, 'inception_score_std': 0.34837200576093874, 'frechet_inception_distance': 2.107874122379201}


## A generated dataset

In [19]:
feed_imgs = np.load('feed_imgs.npy')
feed_imgs2 = feed_imgs.transpose(0, 3, 1, 2)

In [20]:

start = time.time()
px_dataset = TensorDataset(torch.Tensor(feed_imgs2).to(dtype=torch.uint8))

metrics_dict = torch_fidelity.calculate_metrics(
    input1=px_dataset,
    input2=f'cifar10-train',
    cuda=True,
    isc=True,
    fid=True,
    # kid=True,
    verbose=False,
)
print("takes {}s".format(time.time() - start))
print(metrics_dict)

# tensorflow's result
# Obtained fid value of 16.04265464806963
# Inception score of 7.869165420532227
# FID of score 16.04265464806963 takes 168.163743019104s

takes 31.553361177444458s
{'inception_score_mean': 7.75461306602786, 'inception_score_std': 0.19010311746782324, 'frechet_inception_distance': 16.19926870440048}
